# DURATION

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
import scipy
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import statsmodels.formula.api as sm
import copy
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import sklearn
from math import sqrt
from functools import reduce
import numpy as np
#import ee
#import geemap
import geopandas as gpd
import rioxarray as rxr
import rasterstats as rs
from rasterio.plot import show
import rasterio
import cartopy.crs as ccrs
import gdal
%matplotlib inline
plt.rcParams["font.family"] = "Arial"

# IRRIGATED
## NDVI AND OTHER DATA 2001-2018

In [2]:
ndviFilePath='/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/pts_seasonalCompositeNDVI_2001-2018_SUM_JAS.gpkg'
points = gpd.read_file(ndviFilePath)
points = points[(points.Class == 'Irrigated Rice')].groupby(['Class','ADM3_EN']).mean().reset_index()
# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['yr1'] = points.loc[: , 2017:2017].mean(axis=1)
points['yr3'] = points.loc[: , 2014:2017].mean(axis=1)
points['yr5'] = points.loc[: , 2012:2017].mean(axis=1)
points['yr10'] = points.loc[: , 2008:2017].mean(axis=1)
points['yr15'] = points.loc[: , 2003:2017].mean(axis=1)


## ACTUAL DATA 2001-2018

startYear = 2014
endYear = 2017 #2017

actual = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/iloilo_rice_data_cleaned_imputed_new.csv')
actual = actual[(actual.param == 'yield') & (actual.ecosystem == 'irri')].reset_index(drop=True)
actual = actual[(actual.year <= endYear) & (actual.year >= startYear)].groupby('muni').mean().reset_index(drop=False)


merged = pd.merge(actual, points, left_on='muni', right_on='ADM3_EN')

df = pd.DataFrame(merged,columns=['value','yr1',	'yr3',	'yr5',	'yr10','yr15','test'])
df.corr()


,value,yr1,yr3,yr5,yr10,yr15,test
value,1.000000,0.559602,0.643953,0.685701,0.655501,0.644595,NaN
yr1,0.559602,1.000000,0.877406,0.915645,0.891992,0.818549,NaN
yr3,0.643953,0.877406,1.000000,0.972304,0.934850,0.896316,NaN
yr5,0.685701,0.915645,0.972304,1.000000,0.952178,0.884104,NaN
yr10,0.655501,0.891992,0.934850,0.952178,1.000000,0.955159,NaN
yr15,0.644595,0.818549,0.896316,0.884104,0.955159,1.000000,NaN
test,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# RAINFED
## NDVI AND OTHER DATA 2001-2018

In [28]:
ndviFilePath='/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/geospatial_layers/pts_seasonalCompositeNDVI_2001-2018_SUM_JFM.gpkg'
points = gpd.read_file(ndviFilePath)
points = points[(points.Class == 'Rain-fed Rice')].groupby(['Class','ADM3_EN']).mean().reset_index()
# Rename column headings
points = points.rename(columns=dict(zip(list(points.columns[2:20]), list(np.arange(2001, 2019, 1)))))
points['yr1'] = points.loc[: , 2017:2017].mean(axis=1)
points['yr3'] = points.loc[: , 2014:2017].mean(axis=1)
points['yr5'] = points.loc[: , 2012:2017].mean(axis=1)
points['yr10'] = points.loc[: , 2008:2017].mean(axis=1)
points['yr15'] = points.loc[: , 2003:2017].mean(axis=1)
points

## ACTUAL DATA 2001-2018

startYear = 2003
endYear = 2017 #2017

actual = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/iloilo_rice_data_cleaned_imputed_new.csv')
actual = actual[(actual.param == 'yield') & (actual.ecosystem == 'rf')].reset_index(drop=True)
actual = actual[(actual.year <= endYear) & (actual.year >= startYear)].groupby('muni').mean().reset_index(drop=False)


merged = pd.merge(actual, points, left_on='muni', right_on='ADM3_EN')

df = pd.DataFrame(merged,columns=['ADM3_EN','value','yr1',	'yr3',	'yr5',	'yr10','yr15'])
df = df.loc[df['ADM3_EN'].isin([
'Carles',
'Lambunao',
'Calinog',
'Duenas',
'Estancia',
'Sta Barbara',
'Maasin',
'Cabatuan',
'Leon',
'Bingawan',
'Passi City',
'Badiangan',
'San Dionisio',
'Balasan',
'Alimodian',
'Guimbal',
'New Lucena',
'Igbaras',
'Tubungan',
'Janiuay',
'San Enrique',
'Iloilo City',
'Batad' 
])]


df.corr()

,value,yr1,yr3,yr5,yr10,yr15
value,1.000000,0.151975,0.137328,0.196131,0.155008,0.126439
yr1,0.151975,1.000000,0.737282,0.718114,0.615712,0.502829
yr3,0.137328,0.737282,1.000000,0.989274,0.966290,0.929706
yr5,0.196131,0.718114,0.989274,1.000000,0.977820,0.937372
yr10,0.155008,0.615712,0.966290,0.977820,1.000000,0.985227
yr15,0.126439,0.502829,0.929706,0.937372,0.985227,1.000000
